# Bài 2: Gradient Descent và Linear Regression

Trong bài thực hành này, chúng ta sẽ 
- giải linear regression bằng Gradient Descent


## 1. Gradient Descent

In [ ]:
import pandas as pd
df = pd.read_csv("House_Price.csv")        ## load file csv vào pd.DataFrame

In [ ]:
df                 ## xem thử DataFrame

In [ ]:
## vẽ hình
import matplotlib.pyplot as plt
plt.scatter(df["Size in feet squared"], df["Price in thousand dollar"])
plt.xlabel("Size in feet squared")
plt.ylabel("Price in thousand dollar")

Bây giờ chúng ta muốn làm một bài toán dựa vào kích thước của ngôi nhà, đoán giá tiền

- $X$ = diện tích ngôi nhà

- $y$ = giá tiền (chưa biết, muốn dự đoán)

Ta giả thiết $\hat{y} = aX + b$  với $a$ và $b$ là số thực

Ta tìm $a$ và $b$ sao cho trung bình bình phương sai số nhỏ nhất khi đoán, tức là tìm $a$ và $b$ sao cho

$$ L = \frac{1}{n} \sum (y-\hat{y})^2 =  \frac{1}{n} \sum (y - aX - b)^2 $$

đạt giá trị nhỏ nhất, với $n$ là số mẫu.

Đặt $\lambda$ là learning rate, giá trị $a$ và $b$ sẽ được cập nhật theo công thức:

 - $a = a - \lambda \frac{\partial L}{\partial a} = a - \lambda \frac{\partial}{\partial a} \left( \frac{1}{n} \sum (y - aX - b)^2 \right)
= a - 2 \lambda  \frac{1}{n} \sum (y - aX - b) (-X) $

- $b = b - \lambda \frac{\partial L}{\partial b} = b - \lambda \frac{1}{n} \frac{\partial}{\partial b} \left( \sum (y - aX - b)^2 \right)
= b - 2 \lambda  \frac{1}{n} \sum (y - aX - b) (-1) $

Với $X$, $y$ là các ma trận với mỗi hàng là một sample. Tức là trong bài toán này, mỗi hàng của $X$ là diện tích của một căn nhà, tương ứng hàng của $y$ là giá của nhà đó.
Khi đó bài toán Linear Regression có thể code thành
```python
a = a - lmda * 2 *np.mean((y-a*X-b)*(-X))
a = a - lmda * 2 *np.mean((y-a*X-b)*(-1))
```

Cụ thể, bài toán được lập trình như sau

In [ ]:
X = df['Size in feet squared'].values   
X = X.reshape(-1, 1)                   
y = df['Price in thousand dollar'].values 
y = y.reshape(-1,1)


In [ ]:
import numpy as np

a, b = 1.0, 100.0           ## khởi tạo a, b

n_iter = 10000           ## số vòng lặp cập nhật a, b
lmda = 0.0000001          ## learning rate
for ii in range(n_iter):
    error = y - a*X-b
    a = a - 2* lmda * np.mean(error*(-X))    ##cập nhật a và b
    b = b - 2* lmda * np.mean(error*(-1))
    if ii % 1000 == 0:
        loss = np.mean(np.square(y - a*X-b))
        print("Loss at iter {}: {}".format(ii, loss))
loss = np.mean(np.square(y - a*X-b))
print("Final loss: ", loss)

In [ ]:
error.shape, X.shape

In [ ]:
## vẽ hình
import matplotlib.pyplot as plt
plt.scatter(df["Size in feet squared"], df["Price in thousand dollar"])
plt.plot(np.arange(500, 3000), a*np.arange(500, 3000) + b)   ## vẽ solution tìm được
plt.xlabel("Size in feet squared")
plt.ylabel("Price in thousand dollar")

In [ ]:
g_a, g_b = a, b

### Viết gọn hơn xíu

Cho $X$ là vector cột, $Z = (X, 1)$ là ma trận với 2 cột là $X$ và 1, khi đó $\hat{y} = aX + b =  Z \theta$ với $\theta = (a,b)^T$. Bài toán chuyển thành tìm $\theta$ sao cho

   $$ L = \frac{1}{n} ( y - Z \theta )^T ( y - Z \theta )$$
    
nhỏ nhất. Khi đó
$$ \frac{\partial L}{\partial \theta} = \frac{2}{n} (-Z)^T (y-Z \theta)
$$

In [ ]:
X = df['Size in feet squared'].values   
X = X.reshape(-1, 1)                   
y = df['Price in thousand dollar'].values 
y = y.reshape(-1,1)


In [ ]:
import numpy as np
Z = np.concatenate([X, np.ones((X.shape[0],1))], axis=-1) ## thêm một cột toàn số 1

In [ ]:
theta = np.asarray([[1.0], [100.0]])     ## khởi tạo theta

n_iter = 10000           ## số vòng lặp cập nhật theta
lmda = 0.0000001          ## learning rate
for ii in range(n_iter):
    error = y - np.matmul(Z, theta)
    theta =  theta - 2/Z.shape[0]*lmda*np.matmul(-Z.T, error)   ##cập nhật theta
    if ii % 1000 == 0:
        loss = np.mean(np.square(y - np.matmul(Z, theta)))
        print("Loss at iter {}: {}".format(ii, loss))
loss = np.mean(np.square(y - np.matmul(Z,theta))
print("Final loss: ", loss)
print("Final theta: ", theta)

### 1.1 So sánh với exact solution và Sklearn

In [ ]:

## exact solution
import numpy as np
Z = np.concatenate([X, np.ones((100,1))], axis=-1)   
Z = Z.astype(np.float64)       

## giá trị a và b tìm được
T = np.matmul(np.matmul(np.linalg.inv(np.matmul(Z.T,Z)), Z.T), y)
e_a, e_b = T[0][0], T[1][0]
print(e_a, e_b)

In [ ]:
## SkLearn

from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X, y)                ## train LinearRegression 
sk_a, sk_b = lr.coef_[0][0], lr.intercept_[0]    ## lấy các tham số học được
print("a = {}, b = {}".format(sk_a, sk_b))

In [ ]:
print("GD MSE: ", np.mean(np.square(y - g_a*X-g_b)))
print("Exact SOl MSE: ", np.mean(np.square(y - e_a*X-e_b)))
print("SK MSE: ", np.mean(np.square(y - sk_a*X-sk_b)))

In [ ]:
## vẽ hình
import matplotlib.pyplot as plt
plt.scatter(df["Size in feet squared"], df["Price in thousand dollar"])
plt.plot(np.arange(500, 3000), e_a*np.arange(500, 3000) + e_b, label="exact sol") 
plt.plot(np.arange(500, 3000), g_a*np.arange(500, 3000) + g_b, label="GD sol")   
plt.plot(np.arange(500, 3000), sk_a*np.arange(500, 3000) + sk_b, label="SK sol")   
plt.legend()
plt.xlabel("Size in feet squared")
plt.ylabel("Price in thousand dollar")

## Bài tập
Cho dữ liệu gồm 100 hàng, gồm chiều rộng, chiều dài và giá của một ngôi nhà, hay xây dựng một mô hình dư đoán giá nhà dựa vào chiều dài và rộng với giả thuyết
$$ \text{giá} = a * \text{rộng} + b* \text{dài} + c , a, b, c \in \mathbb{R}$$
- Dùng Gradient Descent giải Linear Regression cho bài toán này 
    - khởi tạo tham số, chỉnh số vòng lặp và learning rate cho phù hợp
    - Hint:  để GD dễ hội tụ hơn nên Normalization input trước
- So sánh kết quả (MSE) với Exact Solution và kết quả từ Sklearn

## Nộp bài
- Code và chạy kết quả lưu vào file notebook NMMH_TH2_MSSV.ipynb (notebook phải có kết quả chạy nếu ko xem như chưa làm)
- Nén thành file NMMH_TH2_MSSV.rar (.zip) và nộp về: huynhthanh98vn@gmail.com,  với tiêu đề NMMH_TH2_MSSV
- Deadline: 23g59 thứ 3 ngày 02/06/2020. Nộp trễ bị chia đôi số điểm.

In [ ]:
df = pd.read_csv("House_Price_2.csv")
df